In [1]:
import os

import numpy as np

from keras.models import load_model

import cv2

import pandas as pd

import time

import loss_functions


Using TensorFlow backend.


In [6]:
model_name = 'unet_vanilla_128_adam_binary-crossentropy_iou-metric-64.h5'
model_path = '../models/trained/' + model_name
model_input_size = 101 if '101' in model_path else 128

test_img_path = '../data/test/'
valid_img_path = '../data/{}/all_samples/'.format(model_input_size)
submission_path = '../submissions/'+ model_path.split('/')[-1][:-3] + '.csv'

need_resize = model_input_size != 101

In [7]:
for root, dirs, test_img_names in os.walk(test_img_path):
    test_img_ids = [img_name[:-4] for img_name in test_img_names]
    test_img_paths = [test_img_path + img_name for img_name in test_img_names]

In [8]:
x_valid = np.load(valid_img_path + 'x_valid.npy')
y_valid = np.load(valid_img_path + 'y_valid.npy')

In [17]:
x_valid[i].shape

(128, 128, 1)

In [9]:
model = load_model(model_path, custom_objects={'iou':loss_functions.iou})

W0813 11:42:32.053045  7656 deprecation_wrapper.py:119] From c:\users\robert.colt\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 11:42:32.093052  7656 deprecation_wrapper.py:119] From c:\users\robert.colt\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 11:42:32.129042  7656 deprecation_wrapper.py:119] From c:\users\robert.colt\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 11:42:32.129042  7656 deprecation_wrapper.py:119] From c:\users\robert.colt\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_

In [10]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     print(runs, runs[1::2])
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)



In [12]:
def predict(img, thresh):
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1, model_input_size, model_input_size, 1))
    prediction = model.predict(img).reshape((model_input_size, model_input_size, 1))
#     
    prediction = (prediction > thresh).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
        prediction = prediction >= 0.99
    return prediction

In [15]:
# find best threshhod
thresh_iou = {}
for thresh in np.arange(0.55, 1.0, 0.05):
    avg_iou = 0
    for i, img in enumerate(range(x_valid.shape[0])):
        prediction = model.predict(x_valid[i].reshape(1, 101, 101, 1))
        avg_iou += loss_functions.iou_np((prediction > thresh).astype(int), y_valid[i])
    
    avg_iou = avg_iou / x_valid.shape[0]
    thresh_iou[thresh] = avg_iou
    print(thresh_iou)

ValueError: cannot reshape array of size 16384 into shape (1,101,101,1)

In [ ]:
rle_masks = []
maxcnt = 10
start = time.time()


for i, img_path in enumerate(test_img_paths):
    if i%200 == 0:
        print('{}, {}s'.format(i, time.time() - start))
        start = time.time()
    img = cv2.imread(img_path, 0)/255
    if i == 0:
        print(img.mean())
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1, model_input_size, model_input_size, 1))
    prediction = model.predict(img).reshape((model_input_size, model_input_size, 1))
    prediction = (prediction > 0.7).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
        prediction = prediction >= 0.99
#     prediction = predict(img, 0.74)
    rle_masks.append(rle_encode(prediction))

print('finished 0.89')
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
submission_df.to_csv(submission_path, index=0)

0, 0.0s
0.541629328920079
200, 45.722532510757446s
400, 47.26487445831299s
600, 49.873963594436646s
800, 48.65267205238342s
1000, 50.97232246398926s
1200, 48.60570049285889s
1400, 48.90052914619446s
1600, 48.01656413078308s
1800, 48.086047887802124s
2000, 47.86413049697876s
2200, 48.108805894851685s
2400, 47.3324499130249s
2600, 47.39816665649414s
2800, 50.373905658721924s
3000, 51.796579360961914s
3200, 49.79201579093933s
3400, 49.732089042663574s
3600, 53.188852071762085s
3800, 52.60383081436157s


In [17]:
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
submission_df.to_csv(submission_path, index=0)

In [23]:
submission_df.head()

,id,rle_mask
0,0005bb9630,
1,000a68e46c,
2,000c8dfb2a,
3,000d0a5f6c,
4,001ef8fc87,1 7358 7363 7 7372 79 7475 71 7576 64 7677 58 ...
